In [195]:
import time
import requests
from bs4 import BeautifulSoup
import lxml
from tqdm.notebook import tqdm
import re

from fake_useragent import UserAgent

import undetected_chromedriver 

import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import date

import warnings
warnings.filterwarnings('ignore')

# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
from seleniumwire import webdriver

In [2]:
# указать пользователя
user = 'MerinovDV'
# если файл перенесен в другую папку то сменить путь ниже
path_to_credential = f'C:/Users/{user}/Downloads/auto-monitoring-367212-64ec4ad9d3a5.json' 

In [3]:
# Данные для доступа к Google Spreadsheets

# Specify path to your file with credentials
path_to_credential = path_to_credential 

# Указать название файла в google sheets
table_name = 'yandex_parser' 

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(path_to_credential, scope)

gs = gspread.authorize(credentials)
work_table = gs.open(table_name)


In [4]:
# Получаем список ссылок из google sheets
worksheet = work_table.worksheet('gaming_console')
links_df = worksheet.get_all_values()
columns = links_df.pop(0)
links_df = pd.DataFrame(links_df, columns=columns)

In [5]:
links_df

,Производитель,Модель,Ссылка
0,Sony,PlayStation 5,https://market.yandex.ru/product--igrovaia-pri...
1,Sony,PlayStation 5 Digital Edition,https://market.yandex.ru/product--igrovaia-pri...
2,Nintendo,Switch OLED,https://market.yandex.ru/product--igrovaia-pri...
3,Nintendo,Switch Lite,https://market.yandex.ru/product--igrovaia-pri...
4,Nintendo,Switch rev.2,https://market.yandex.ru/product--igrovaia-pri...
5,Microsoft,Xbox Series X,https://market.yandex.ru/product--igrovaia-pri...
6,Microsoft,Xbox Series S,https://market.yandex.ru/product--igrovaia-pri...


In [6]:
def open_title_link(url):
    
    """ 
    функция вовзвращает объект soup по заданной ссылке на титульную страуницу товара
    
    url: str
        ссылка на титульную страницу товара
    """
    ua = UserAgent()
    user_agent = ua.random
    
    options = webdriver.ChromeOptions()
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(
        executable_path=f"C:/Users/{user}/Parsing_folder/chromedriver/chromedriver.exe",
        options=options)
    
    try:
        driver.get(url)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'lxml')
    except Exception as ex:
        print(ex)
    finally:
        driver.close()
        driver.quit()
        
    return soup

In [8]:
soup = open_link(links_df['Ссылка'][0])

In [7]:
def get_info_title(soup):
    
    """ 
    функция вовзвращает информацию о товаре с его титульнойстраницы
    soup: BeautifulSoup object
        объект soup титульной страницы товара
    """
    
    # получасем кол-во человек, которые интересовались товаром за последние 2 месяца
    interest_badge_text = soup.find(attrs={'data-zone-name': "interest-badge"}).getText().replace(u'\xa0', u'')
    interest_badge_num = int(re.findall('\d+', interest_badge_text)[0])
    
    # получаем данные по рейтингу товара
    rating_dict = {}
    rating_meta = soup.find('div', attrs={'itemprop': "aggregateRating"}).find_all('meta')
    for item in rating_meta:
        rating_dict[item['itemprop']] = item['content']
    
    # получаем кол-во предложений и минимальная цена
    offers_minprice = soup.find('div', attrs={'data-zone-name': "morePricesLink"}).getText().replace(' ', '')
    offers, min_price = re.findall('\d+', offers_minprice)
    offers = int(offers)
    min_price = int(min_price)
    
    title_info_dict = {
        'интересовались за последние 2 месяца': interest_badge_num,
        'рейтинг': rating_dict, 
        'кол-во предложений': offers,
        'минимальная цена': min_price
    }
    
    return title_info_dict

In [54]:
title_info = get_info_title(soup)
title_info

{'интересовались за последние 2 месяца': 208397,
 'рейтинг': {'ratingValue': '4.5',
  'bestRating': '5',
  'worstRating': '1',
  'ratingCount': '3389',
  'reviewCount': '1016'},
 'кол-во предложений': 141,
 'минимальная цена': 51124}

In [210]:
def open_offers_link(url):
    
    """ 
    функция вовзвращает объект soup по заданной ссылке на страницу с предложениями по товару
    
    url: str
        ссылка на страницу с предложениями по товару
    """
    
    ua = UserAgent()
    user_agent = ua.random
    
    options = webdriver.ChromeOptions()
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    driver = webdriver.Chrome(
        executable_path=f"C:/Users/{user}/Parsing_folder/chromedriver/chromedriver.exe",
        options=options)
    
    def get_offers_url(url):

        """функция возвращает ссылку на список предложений по товару
        url: str
             ссылка на страницу товара в Яндекс Маркете
        """

        splited_url = url.split('/')
        item_filter = splited_url.pop()
        splited_url.append('/offers?'.join(item_filter.split('?')))
        offers_url = '/'.join(splited_url) + '&how=aprice'

        return offers_url
    
    offers_url = get_offers_url(url)
    
    try:
        driver.get(offers_url)
        time.sleep(10)
        soup = BeautifulSoup(driver.page_source, 'lxml')
    except Exception as ex:
        print(ex)
    finally:
        driver.close()
        driver.quit()
        
    return soup

In [230]:
def get_info_offers(soup):
    
    """ 
    функция вовзвращает информацию о предложениях товара
    
    soup: BeautifulSoup object
        объект soup страницы с предложениями товара
    """
    offers_info_dict = {}
    
    # получасем название магазина и цену
    snippet_list = soup.find('div', attrs={'data-zone-name': "snippetList"})
    
    for i in range(len(snippet_list)):
        offer_snippet = snippet_list.find_all('div', attrs={'data-zone-name': "OfferSnippet"})[i]\
            .find('div', attrs={'data-auto':'top-offer-snippet-shop-info'})
        
        if offer_snippet.find('div', attrs={'data-zone-name':'shop-name'}) is not None:
            shop_name_div = offer_snippet.find('div', attrs={'data-zone-name':'shop-name'})
            try:
                shop_name = shop_name_div.find('img')['title']
            except:
                shop_name = shop_name_div.find('a')['title']

        elif offer_snippet.find('a', attrs={'data-zone-name':'offerLink'}) is not None:
            shop_name = offer_snippet.find('a', attrs={'data-zone-name':'offerLink'}).getText()
        
        else:
            shop_name = 'unknown'
        
        price_snippet = snippet_list.find_all('div', attrs={'data-zone-name': "price"})[i]\
            .find('span', attrs={'data-auto':'mainPrice'})
        
        price = price_snippet.getText().replace(u'\xa0', u'').replace('₽', '').replace(' ', '')
        
        offers_info_dict[shop_name] = int(price)
    
    return offers_info_dict

In [212]:
soup = open_offers_link(links_df['Ссылка'][0])

In [231]:
get_info_offers(soup)

{'ID Store': 50850,
 'GOODSTORE.COM': 50889,
 'NAVITOYS.RU': 50890,
 'СОТОВИКmobile': 51970,
 'MOBILE-ROOMS.RU': 52150,
 'СОТОХИТ.РФ': 53350,
 'www.video-shoper.ru': 53590,
 'OFFO.RU': 54580,
 'Магазин iCases.Ru': 54990,
 'SONY CENTRE': 54999}